In [1]:
from datetime import date

import pandas as pd
import polars as pl
import polars.selectors as cs

data = {
    "col1": [1, 2, 3],
    "col2": [4.1, 5.2, 6.3],
    "col3": ["x", "y", "z"],
    "col4": pl.date_range(
        date(2022, 1, 1), date(2022, 3, 1), "1mo", eager=True
    ).alias("date"),
    "col5": [True, True, True],
}
df = pl.DataFrame(data)
print(df)

shape: (3, 5)
┌──────┬──────┬──────┬────────────┬──────┐
│ col1 ┆ col2 ┆ col3 ┆ col4       ┆ col5 │
│ ---  ┆ ---  ┆ ---  ┆ ---        ┆ ---  │
│ i64  ┆ f64  ┆ str  ┆ date       ┆ bool │
╞══════╪══════╪══════╪════════════╪══════╡
│ 1    ┆ 4.1  ┆ x    ┆ 2022-01-01 ┆ true │
│ 2    ┆ 5.2  ┆ y    ┆ 2022-02-01 ┆ true │
│ 3    ┆ 6.3  ┆ z    ┆ 2022-03-01 ┆ true │
└──────┴──────┴──────┴────────────┴──────┘


## Expr

In [2]:
print(df.select(pl.col("col1").add(1)))

shape: (3, 1)
┌──────┐
│ col1 │
│ ---  │
│ i64  │
╞══════╡
│ 2    │
│ 3    │
│ 4    │
└──────┘


In [3]:
col1_add_one = pl.col("col1").add(1)

print(df.select(col1_add_one))

shape: (3, 1)
┌──────┐
│ col1 │
│ ---  │
│ i64  │
╞══════╡
│ 2    │
│ 3    │
│ 4    │
└──────┘


In [4]:
def add_one(col: str, alias: str | None = None) -> pl.Expr:
    expr = pl.col(col).add(1)
    if alias is not None:
        expr = expr.alias(alias)
    return expr


print(df.select(add_one("col1")))

shape: (3, 1)
┌──────┐
│ col1 │
│ ---  │
│ i64  │
╞══════╡
│ 2    │
│ 3    │
│ 4    │
└──────┘


### Expr in contexts

In [5]:
col3_eq_y = pl.col("col3").eq("y")

In [6]:
print(df.select(col3_eq_y))

shape: (3, 1)
┌───────┐
│ col3  │
│ ---   │
│ bool  │
╞═══════╡
│ false │
│ true  │
│ false │
└───────┘


In [7]:
print(df.filter(col3_eq_y))

shape: (1, 5)
┌──────┬──────┬──────┬────────────┬──────┐
│ col1 ┆ col2 ┆ col3 ┆ col4       ┆ col5 │
│ ---  ┆ ---  ┆ ---  ┆ ---        ┆ ---  │
│ i64  ┆ f64  ┆ str  ┆ date       ┆ bool │
╞══════╪══════╪══════╪════════════╪══════╡
│ 2    ┆ 5.2  ┆ y    ┆ 2022-02-01 ┆ true │
└──────┴──────┴──────┴────────────┴──────┘


In [8]:
print(df.group_by(col3_eq_y).agg(pl.all()))

shape: (2, 5)
┌───────┬───────────┬────────────┬──────────────────────────┬──────────────┐
│ col3  ┆ col1      ┆ col2       ┆ col4                     ┆ col5         │
│ ---   ┆ ---       ┆ ---        ┆ ---                      ┆ ---          │
│ bool  ┆ list[i64] ┆ list[f64]  ┆ list[date]               ┆ list[bool]   │
╞═══════╪═══════════╪════════════╪══════════════════════════╪══════════════╡
│ false ┆ [1, 3]    ┆ [4.1, 6.3] ┆ [2022-01-01, 2022-03-01] ┆ [true, true] │
│ true  ┆ [2]       ┆ [5.2]      ┆ [2022-02-01]             ┆ [true]       │
└───────┴───────────┴────────────┴──────────────────────────┴──────────────┘


## Selector

In [9]:
print(df.select(cs.integer()))

shape: (3, 1)
┌──────┐
│ col1 │
│ ---  │
│ i64  │
╞══════╡
│ 1    │
│ 2    │
│ 3    │
└──────┘


In [10]:
from polars.datatypes.group import INTEGER_DTYPES

print(df.select(pl.col(INTEGER_DTYPES)))

shape: (3, 1)
┌──────┐
│ col1 │
│ ---  │
│ i64  │
╞══════╡
│ 1    │
│ 2    │
│ 3    │
└──────┘


In [11]:
print(df.select(cs.temporal()))

shape: (3, 1)
┌────────────┐
│ col4       │
│ ---        │
│ date       │
╞════════════╡
│ 2022-01-01 │
│ 2022-02-01 │
│ 2022-03-01 │
└────────────┘


In [12]:
print(df.select(cs.numeric()))

shape: (3, 2)
┌──────┬──────┐
│ col1 ┆ col2 │
│ ---  ┆ ---  │
│ i64  ┆ f64  │
╞══════╪══════╡
│ 1    ┆ 4.1  │
│ 2    ┆ 5.2  │
│ 3    ┆ 6.3  │
└──────┴──────┘


## Union

In [13]:
print(df.select(cs.numeric() | cs.string()))

shape: (3, 3)
┌──────┬──────┬──────┐
│ col1 ┆ col2 ┆ col3 │
│ ---  ┆ ---  ┆ ---  │
│ i64  ┆ f64  ┆ str  │
╞══════╪══════╪══════╡
│ 1    ┆ 4.1  ┆ x    │
│ 2    ┆ 5.2  ┆ y    │
│ 3    ┆ 6.3  ┆ z    │
└──────┴──────┴──────┘


## Intersection

In [14]:
print(df.select(cs.numeric() & cs.by_name("col2", "col5")))

shape: (3, 1)
┌──────┐
│ col2 │
│ ---  │
│ f64  │
╞══════╡
│ 4.1  │
│ 5.2  │
│ 6.3  │
└──────┘


## Difference

In [15]:
print(df.select(cs.numeric() - cs.integer()))

shape: (3, 1)
┌──────┐
│ col2 │
│ ---  │
│ f64  │
╞══════╡
│ 4.1  │
│ 5.2  │
│ 6.3  │
└──────┘


## Symmetric difference

In [16]:
print(df.select(cs.numeric() ^ (cs.by_name("col2", "col5"))))

shape: (3, 2)
┌──────┬──────┐
│ col1 ┆ col5 │
│ ---  ┆ ---  │
│ i64  ┆ bool │
╞══════╪══════╡
│ 1    ┆ true │
│ 2    ┆ true │
│ 3    ┆ true │
└──────┴──────┘


## Complement

In [17]:
print(df.select(~cs.temporal()))

shape: (3, 4)
┌──────┬──────┬──────┬──────┐
│ col1 ┆ col2 ┆ col3 ┆ col5 │
│ ---  ┆ ---  ┆ ---  ┆ ---  │
│ i64  ┆ f64  ┆ str  ┆ bool │
╞══════╪══════╪══════╪══════╡
│ 1    ┆ 4.1  ┆ x    ┆ true │
│ 2    ┆ 5.2  ┆ y    ┆ true │
│ 3    ┆ 6.3  ┆ z    ┆ true │
└──────┴──────┴──────┴──────┘


## `+` is not working

In [18]:
# df.select(cs.numeric() + cs.string())

## as_expr()

In [19]:
# Wrong
print(df.select(~cs.boolean()))

shape: (3, 4)
┌──────┬──────┬──────┬────────────┐
│ col1 ┆ col2 ┆ col3 ┆ col4       │
│ ---  ┆ ---  ┆ ---  ┆ ---        │
│ i64  ┆ f64  ┆ str  ┆ date       │
╞══════╪══════╪══════╪════════════╡
│ 1    ┆ 4.1  ┆ x    ┆ 2022-01-01 │
│ 2    ┆ 5.2  ┆ y    ┆ 2022-02-01 │
│ 3    ┆ 6.3  ┆ z    ┆ 2022-03-01 │
└──────┴──────┴──────┴────────────┘


In [20]:
print(df.select(~(cs.boolean().as_expr())))

shape: (3, 1)
┌───────┐
│ col5  │
│ ---   │
│ bool  │
╞═══════╡
│ false │
│ false │
│ false │
└───────┘


## Ref

In [21]:
print(
    pl.DataFrame(
        {
            "Operator": ["A | B", "A & B", "A - B", "A ^ B", "~A"],
            "Operation": [
                "Union",
                "Intersection",
                "Difference",
                "Symmetric difference",
                "Complement",
            ],
        }
    )
)

shape: (5, 2)
┌──────────┬──────────────────────┐
│ Operator ┆ Operation            │
│ ---      ┆ ---                  │
│ str      ┆ str                  │
╞══════════╪══════════════════════╡
│ A | B    ┆ Union                │
│ A & B    ┆ Intersection         │
│ A - B    ┆ Difference           │
│ A ^ B    ┆ Symmetric difference │
│ ~A       ┆ Complement           │
└──────────┴──────────────────────┘


## codepanda

In [22]:
df_pd = pd.DataFrame(data)

print(df_pd.select_dtypes("number"))

   col1  col2
0     1   4.1
1     2   5.2
2     3   6.3


In [23]:
print(df_pd.filter(regex="^col(?!4$|5$)", axis=1))

   col1  col2 col3
0     1   4.1    x
1     2   5.2    y
2     3   6.3    z
